# Imports

In [13]:
import os
from typing import Dict, List, Tuple

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [14]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 60                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data"
DEPRESJON_PREFIX = "manual_depresjon"
PSYKOSE_PREFIX = "manual_psykose"
HYPERAKTIV_PREFIX = "manual_hyperaktiv"
MAIN_RESULTS_DIR = "results"

# Manual feature extraction

## Helper functions

In [15]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - Format "timestamp" to YYYY-MM-DD HH:MM:SS
    - Drop redundant "date" column
    - Convert "activity" to float32
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        # Convert and enforce the desired timestamp format
        df["timestamp"] = pd.to_datetime(df["timestamp"], dayfirst=False)
        df["timestamp"] = df["timestamp"].dt.strftime("%Y-%m-%d %H:%M:%S")
        
        # Drop "date" column if it exists
        if "date" in df.columns:
            df.drop("date", axis=1, inplace=True)
        
        # Ensure "activity" column is float32
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame, freq: str = "min") -> pd.DataFrame:
    """
    Fill missing activity values by resampling based on given frequency.
    
    :param df: DataFrame with 'timestamp' and 'activity' columns.
    :param freq: Resampling frequency (default: minute).
    :return: DataFrame with missing values filled.
    """
    df = df.copy() # create copy to avoid side effects
  
    df["timestamp"] = pd.to_datetime(df["timestamp"])

    df.set_index("timestamp", inplace=True)

    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample(freq).mean()
    
    # recreate index and "timestamp" column
    df.reset_index(inplace=True)

    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())
    
    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """

    activity = df["activity"].values
    nperseg = min(NPERSEG, len(activity))  # Ensure nperseg doesn't exceed data length
    noverlap = int(0.75 * nperseg) 
    
    psd = scipy.signal.welch(
        x=activity,
        fs=(1/60),
        nperseg=nperseg,
        noverlap=noverlap,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """

    activity = df["activity"].values
    nperseg = min(NPERSEG, len(activity))  # Ensure nperseg doesn't exceed data length
    noverlap = int(0.75 * nperseg) 

    power_spectrum = scipy.signal.welch(
        activity,
        fs=(1/60),
        nperseg=nperseg,
        noverlap=noverlap,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [16]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [17]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [18]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 24hs
    - days: [8:00, 21:00)
    - nights: [21:00, 8:00)
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "full_24h", "day" and "night", corresponding
    to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]
    
    night_dfs = [get_day_part(df, part="night") for df in full_dfs]
    day_dfs = [get_day_part(df, part="day") for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("full_24h", full_dfs), ("night", night_dfs),
                              ("day", day_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets

## Hyperaktiv

In [19]:
dataset = Dataset(dirpath=os.path.join("data", "hyperaktiv"),sep=';')
condition = dataset.condition
control = dataset.control

In [20]:
condition[0]

,TIMESTAMP,ACTIVITY
0,02-23-2009 16:00,0
1,02-23-2009 16:01,195
2,02-23-2009 16:02,240
3,02-23-2009 16:03,209
4,02-23-2009 16:04,202
...,...,...
8631,03-01-2009 15:51,0
8632,03-01-2009 15:52,0
8633,03-01-2009 15:53,0
8634,03-01-2009 15:54,131


In [21]:
for cond in condition:
    cond.columns = cond.columns.str.lower()

for contr in control:
    contr.columns = contr.columns.str.lower()

In [22]:
condition = basic_data_cleaning(condition)
control = basic_data_cleaning(control)

In [23]:
condition[0]

,timestamp,activity
0,2009-02-23 16:00:00,0.0
1,2009-02-23 16:01:00,195.0
2,2009-02-23 16:02:00,240.0
3,2009-02-23 16:03:00,209.0
4,2009-02-23 16:04:00,202.0
...,...,...
8631,2009-03-01 15:51:00,0.0
8632,2009-03-01 15:52:00,0.0
8633,2009-03-01 15:53:00,0.0
8634,2009-03-01 15:54:00,131.0


In [24]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [25]:
entire_df

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,0.000000,403.383331,52.376953,24.858334,5803.127441,6.555398,2.453568,1.445070,57.704167,36.219048,...,1.115775e+06,5.117809e+05,1.770168e+12,1.584219,1.590545,1.192424,1.365035e+06,8.614056e+05,4.118326,0.502390
1,0.000000,693.566650,247.361588,232.966660,33204.535156,-0.569249,0.476879,0.732509,269.033340,236.239960,...,3.628402e+06,1.681671e+06,1.352569e+13,0.360673,1.208220,1.013595,4.614595e+06,2.997240e+06,4.284431,0.560183
2,10.416667,1578.366699,704.924377,694.316650,110641.320312,-0.330919,0.257797,0.469425,455.949982,695.192200,...,1.231488e+07,8.869196e+06,1.763829e+14,12.204724,3.360065,1.078445,9.077075e+06,9.496880e+06,4.427767,0.727455
3,0.000000,226.199997,69.596909,65.563095,2366.400635,0.212139,0.553014,0.694349,61.591667,66.751862,...,3.098033e+05,1.723504e+05,1.432631e+11,3.838823,2.167220,1.221746,1.836900e+05,2.204031e+05,4.184605,0.575401
4,0.000000,777.283325,241.027008,227.016663,41039.054688,0.231119,0.837726,0.833338,258.233343,218.941727,...,4.291866e+06,3.113292e+06,1.775631e+13,5.565704,2.188997,0.981816,3.814352e+06,3.516143e+06,4.400035,0.639341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,11.066667,917.349976,399.441406,401.500000,58780.964844,-0.892120,0.042388,0.603344,364.233322,394.895599,...,6.962186e+06,2.628976e+06,7.453306e+13,1.648226,1.525731,1.240021,1.065506e+07,5.382362e+06,4.008649,0.420595
81,0.000000,1186.816650,443.041931,437.825012,110803.664062,-0.957278,0.412996,0.747147,494.708317,421.392700,...,1.447666e+07,6.347388e+06,3.165527e+14,2.626781,1.838580,1.229008,1.327290e+07,1.075903e+07,4.117688,0.529651
82,5.766667,803.549988,305.222443,282.908325,42047.171875,-0.115300,0.627264,0.667901,261.062504,288.349426,...,3.888262e+06,3.277940e+06,9.851383e+12,5.469484,1.944749,0.807222,3.805199e+06,3.509409e+06,4.546244,0.713455
83,5.500000,1052.777832,381.603149,345.049988,59097.859375,-0.130226,0.638574,0.633541,328.183311,364.508820,...,6.479886e+06,3.791345e+06,3.978240e+13,1.824630,1.620994,0.973370,6.391509e+06,5.268898e+06,4.397201,0.667582


In [26]:
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)
for part, df in datasets.items():
    filename = f"{HYPERAKTIV_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [27]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"hyperaktiv_y.csv")
y.to_csv(filepath, header=False, index=False)

In [28]:
datasets["full_24h"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,0.000000,403.383331,50.180340,23.741667,4793.352051,6.019871,2.292614,1.374905,57.762501,35.359364,...,5.669422e+05,1.681332e+05,5.718588e+11,1.187466,1.581844,1.333846,5.772311e+05,4.051208e+05,3.931571,0.429810
1,0.000000,693.566650,160.376526,96.858337,28786.158203,0.541218,1.164144,1.054724,238.983324,134.976639,...,3.618312e+06,1.701820e+06,3.616424e+13,7.005394,2.868262,1.662010,1.458715e+06,1.925778e+06,3.832459,0.503856
2,0.000000,1578.366699,504.314026,466.516663,160490.796875,-0.750322,0.429993,0.792099,734.025002,473.657410,...,1.825557e+07,9.257409e+06,1.007599e+15,9.173212,3.238597,1.738795,1.009346e+07,1.022590e+07,3.816579,0.510775
3,0.000000,226.199997,60.239891,50.483334,2112.999023,0.265228,0.754047,0.760380,69.679167,56.286095,...,2.741842e+05,1.547824e+05,1.414058e+11,7.934178,2.914684,1.371485,1.625213e+05,1.806811e+05,4.130993,0.600406
4,0.000000,777.283325,181.608994,145.633331,30638.595703,1.930784,1.418712,0.959132,224.033339,154.406952,...,2.440261e+06,1.607829e+06,7.078795e+12,6.708076,2.484032,1.090293,1.695636e+06,1.896236e+06,4.334279,0.631657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,11.066667,917.349976,397.644592,390.649994,44899.832031,-0.622728,0.026176,0.531167,279.191681,396.944366,...,4.927708e+06,1.895514e+06,2.781562e+13,0.159596,1.160468,1.070285,7.288324e+06,4.136616e+06,4.216035,0.543653
81,0.000000,1186.816650,312.927368,197.291656,97797.250000,-0.185516,0.915782,0.996227,496.725019,269.361938,...,1.375517e+07,4.421307e+06,7.522237e+14,8.971806,3.203207,1.993919,5.448047e+06,6.441831e+06,3.508455,0.417335
82,1.866667,803.549988,201.853241,155.683334,42115.000000,0.339172,0.989917,1.013553,315.008333,172.253769,...,4.174413e+06,2.095294e+06,6.921705e+13,10.157345,3.451809,1.993017,1.608000e+06,2.078599e+06,3.631362,0.488241
83,3.100000,1052.777832,258.540192,214.066666,62757.687500,0.135810,0.889887,0.966071,393.779162,226.261353,...,7.327457e+06,3.371278e+06,1.417155e+14,7.568385,2.936304,1.624634,2.200796e+06,4.197837e+06,3.882889,0.529696


# Classification

## Classifiers, parameters, constants

In [29]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=5000
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=500,
        criterion="entropy"
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": [None, "balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    },
    "SVM": {
        "C": np.logspace(10e-3, 10e3, num=50),
        "gamma": np.logspace(10e-3, 10e3, num=50),
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    }
}

## Hyperaktiv Classification

In [44]:
dataset = HYPERAKTIV_PREFIX
y_filename = "hyperaktiv_y.csv"

In [45]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [47]:
sum(y), len(y) - sum(y)

(np.int64(45), np.int64(40))

In [48]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, "hyperactiv")

# for part in ["full_24h", "night", "day"]:
for part in ["night"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    
    
    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"test_scores_{part}_{clf_type}"), index=False)
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"final_scores_{part}_{clf_type}"), index=False)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

PART: night
  LR


KeyboardInterrupt: 

## Depresjon

In [32]:
dataset = Dataset(dirpath=os.path.join("data", "depresjon"))
condition = dataset.condition
control = dataset.control

In [33]:
condition[0]

,timestamp,date,activity
0,2003-05-07 12:00:00,2003-05-07,0
1,2003-05-07 12:01:00,2003-05-07,143
2,2003-05-07 12:02:00,2003-05-07,0
3,2003-05-07 12:03:00,2003-05-07,20
4,2003-05-07 12:04:00,2003-05-07,166
...,...,...,...
23239,2003-05-23 15:19:00,2003-05-23,0
23240,2003-05-23 15:20:00,2003-05-23,0
23241,2003-05-23 15:21:00,2003-05-23,0
23242,2003-05-23 15:22:00,2003-05-23,0


In [34]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [35]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [36]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [37]:
datasets["full_24h"]

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,0.000000,1775.400024,146.843735,79.099998,39430.636719,16.872656,3.217538,1.350520,214.308335,112.766991,...,4.922578e+06,3.096667e+06,6.520324e+13,8.278364,3.055725,1.640371,3.321370e+06,2.812564e+06,3.862526,0.493157
1,0.000000,1184.533325,289.170868,261.433319,73694.851562,-0.232590,0.710244,0.937476,457.241667,258.745453,...,8.541842e+06,2.765303e+06,3.341648e+14,9.678225,3.353926,2.140074,2.754712e+06,3.691799e+06,3.411010,0.420931
2,0.000000,950.250000,129.167282,50.816666,27766.009766,2.877282,1.628733,1.288362,204.808334,98.616768,...,2.768503e+06,1.349840e+06,2.141066e+13,9.650369,3.283605,1.671360,1.286006e+06,1.512580e+06,3.913935,0.567943
3,0.000000,1059.316650,151.091476,81.733337,29531.572266,1.933566,1.321734,1.135836,252.179164,123.827675,...,3.111768e+06,1.344710e+06,3.218593e+13,9.442859,3.299121,1.823164,9.698628e+05,1.655660e+06,3.756400,0.515400
4,0.000000,906.483337,221.277527,178.983337,46180.472656,-0.362119,0.730967,0.970038,362.500005,196.210083,...,4.731094e+06,1.927837e+06,4.484004e+13,6.700033,2.714890,1.415375,3.136303e+06,3.023670e+06,4.061218,0.575490
5,0.000000,719.416687,75.061485,42.591667,8648.376953,9.114863,2.439158,1.237227,102.891665,58.222412,...,7.514579e+05,4.813648e+05,8.108180e+11,9.105030,3.216822,1.198276,1.483000e+05,5.251938e+05,4.363182,0.735476
6,0.000000,1099.250000,109.481110,52.458332,20248.443359,9.519291,2.511456,1.297954,161.033337,82.409325,...,2.071863e+06,1.508277e+06,2.924200e+12,6.016092,2.487304,0.825358,7.576488e+05,1.686145e+06,4.599359,0.805218
7,0.000000,1501.800049,146.327271,5.541667,54861.113281,3.348586,1.840380,1.599542,243.775002,96.446457,...,4.683842e+06,1.973006e+06,8.355921e+13,10.004759,3.417408,1.951618,2.173964e+06,2.364248e+06,3.660217,0.496254
8,0.000000,1046.016724,85.663643,69.050003,8097.554199,34.586815,3.794976,1.049002,118.437495,74.442741,...,7.317696e+05,4.713816e+05,7.760582e+11,7.648185,2.943203,1.203851,2.483017e+05,5.019477e+05,4.325404,0.706048
9,0.000000,389.283325,70.900734,64.666664,3598.278809,4.015282,1.524170,0.844865,78.016665,63.697311,...,3.867453e+05,2.990933e+05,1.105551e+11,1.241426,1.435450,0.859734,2.978725e+05,3.294995e+05,4.493496,0.698339


## Depresjon classification

In [38]:
dataset = DEPRESJON_PREFIX
y_filename = "depresjon_y.csv"

In [39]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [40]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, "depresjon")

# for part in ["full_24h", "night", "day"]:
for part in ["night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    
    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"test_scores_{part}_{clf_type}"), index=False)
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"final_scores_{part}_{clf_type}"), index=False)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

PART: night
  LR


KeyboardInterrupt: 

## Psykose

In [42]:
dataset = Dataset(dirpath=os.path.join("data", "psykose"))
condition = dataset.condition
control = dataset.control

In [43]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["full_24h", "night", "day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    datasets[part] = entire_df

In [104]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [105]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)

## Psykose classification

In [106]:
dataset = PSYKOSE_PREFIX
y_filename = "psykose_y.csv"

In [107]:
datasets = {}

for part in ["full_24h", "night", "day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()

In [12]:
results_directory = os.path.join(".", MAIN_RESULTS_DIR, "psykose")

# for part in ["full_24h", "night", "day"]:
for part in ["night", "day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    
    for clf_type in ["LR", "SVM", "RF"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)
        
        test_scores = []
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="accuracy",
                n_jobs=-1,
                refit=True,
                cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)
            
            clf = grid_search.best_estimator_
            
            metrics = calculate_metrics(clf, X_test, y_test)
            print(metrics)
            test_scores.append(metrics)
        
        pd.DataFrame.from_records(test_scores).to_csv(os.path.join(results_directory, f"test_scores_{part}_{clf_type}"), index=False)
        final_scores = calculate_metrics_statistics(test_scores)
        df = pd.DataFrame([(key,) + values for key, values in final_scores.items()],
                      columns=['Index', 'Mean', 'Stddev']).set_index('Index')
        df.to_csv(os.path.join(results_directory, f"final_scores_{part}_{clf_type}"), index=False)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

PART: night
  LR
{'accuracy': 0.5172413793103449, 'balanced_accuracy': np.float64(0.5), 'f1': np.float64(0.6818181818181818), 'precision': np.float64(0.5172413793103449), 'recall': np.float64(1.0), 'specificity': np.float64(0.0), 'ROC_AUC': np.float64(0.5), 'MCC': np.float64(0.0)}
{'accuracy': 0.4642857142857143, 'balanced_accuracy': np.float64(0.5), 'f1': np.float64(0.0), 'precision': np.float64(1.0), 'recall': np.float64(0.0), 'specificity': np.float64(1.0), 'ROC_AUC': np.float64(0.5), 'MCC': np.float64(0.0)}
{'accuracy': 0.4642857142857143, 'balanced_accuracy': np.float64(0.46923076923076923), 'f1': np.float64(0.4444444444444444), 'precision': np.float64(0.5), 'recall': np.float64(0.4), 'specificity': np.float64(0.5384615384615384), 'ROC_AUC': np.float64(0.4692307692307692), 'MCC': np.float64(-0.062017367294604234)}
    accuracy: 0.4819 +- 0.0250
    balanced_accuracy: 0.4897 +- 0.0145
    f1: 0.3754 +- 0.2826
    precision: 0.6724 +- 0.2317
    recall: 0.4667 +- 0.4110
    specific